In [1]:
import numpy as np
import networkx as nx
import json
import copy
import sys
import os
import random
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Layout, HTML, FloatSlider, Output
import time
import threading
from ipywidgets import interact, interact_manual, fixed
from pythreejs import *
import math
sys.path.insert(0, os.path.abspath('..'))

In [2]:
class CubeBase:
    tables = None
    piece_initial_ids_at_positions = np.array([
        [[1 , 2 , 3 ],
         [4 , 5 , 6 ],
         [7 , 8 , 9 ]], # Front face

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]], # Middle slice

        [[19, 20, 21],
         [22, 23, 24],
         [25, 26, 27]], # Back face
    ])

    cube_faces = {
        'X': piece_initial_ids_at_positions[:,:,2],
        'x': piece_initial_ids_at_positions[:,:,0],
        'Y': piece_initial_ids_at_positions[2,:,:],
        'y': piece_initial_ids_at_positions[0,:,:],
        'Z': piece_initial_ids_at_positions[:,0,:],
        'z': piece_initial_ids_at_positions[:,2,:]
    }
    # So, front face, y,  is piece_initial_positions[0,:,:]
    # and the middle slice is piece_initial_positions[1,:,:]
    # and the back face, Y, is piece_initial_positions[2,:,:]
    # the top face, Z, is piece_initial_positions[:,0,:]
    # the equatorial slice is piece_initial_positions[:,1,:]
    # and the bottom face, z, is piece_initial_positions[:,2,:]
    # the left face, x,  is piece_initial_positions[:,:,0]
    # the sagittal slice is piece_initial_positions[:,:,1]
    # and the right face, X,  is piece_initial_positions[:,:,2]
    # the cube center is piece_initial_positions[1,1,1]

    piece_initial_orientations = np.array([
        [['xyZ', 'g', 'XyZ'],
         ['g'  , 'y', 'g'  ],
         ['xyz', 'g', 'Xyz']],

        [['g'  , 'Z', 'g'  ],
         ['x'  , 'C', 'X'  ],
         ['g'  , 'z', 'g'  ]],

        [['xYZ', 'g', 'XYZ'],
         ['g'  , 'Y', 'g'  ],
         ['xYz', 'g', 'XYz']],
    ]) 
    # for corners, capital letters refer point to positive axes and lower letters point to negative axes.
    # so, for the first corner piece (0,0,0), whose orientation is xyZ, the piece's local coordiantes' x-axis is aligned with the cube's negative x (given by x at the start), y-axis with the cube's negative y (y), and z-axis with the cube's positive z (Z).
    # for edges, 'g' (good) means that the minimum-length path from the current position to the initial position (solved state) solves the edge in isolation
    # and 'b' (bad) means that the minimum-length path from the current position to the initial position (solved state) does not solve the edge in isolation. The solution in isolation is what is called the secondary path: path that is immediately of the next length than the minimum-length path to the solved state.
    # the fourteenth piece (the cube center), marked by 'C', is irrelevant to the program.
    # the centers of the faces are marked by the respective axes that align with their respective perpendiculars.

    @classmethod
    def initialize(cls):
        if cls.tables is None:
            cls.edge_positions, cls.corner_positions = cls.categorize_positions_over_piece_types()
            cls.edge_ids, cls.corner_ids = cls.categorize_ids_over_piece_types()

            cls.edge_positions.sort()
            cls.corner_positions.sort()
            cls.edge_ids.sort()
            cls.corner_ids.sort()

            cls.tables = cls._load_tables_from_json([
                    '../Precomputed_Tables/corner_position_distance_table.json',
                    '../Precomputed_Tables/edge_position_distance_table.json',
                    '../Precomputed_Tables/position_movement_table.json'
            ])
            cls.edge_distances = cls.tables["edge_distances"]
            cls.corner_distances = cls.tables["corner_distances"]
            cls.movements = cls.tables["movements"]

            cls.initial_color_map = {
                # maps peice_ids to their facelets to their respective colors

                # Top face color: White
                # Bottom face color: Yellow
                # Left face color: Orange
                # Right face color: Red
                # Front face color: Green
                # Back face color: Blue

                # Front face (i=0) - IDs 1-9
                1:  {'Z': 'W', 'x': 'O', 'y': 'G'},  # ULF corner
                2:  {'Z': 'W', 'y': 'G'},            # UF edge
                3:  {'Z': 'W', 'X': 'R', 'y': 'G'},  # URF corner
                4:  {'x': 'O', 'y': 'G'},            # LF edge
                5:  {'y': 'G'},                      # F center
                6:  {'X': 'R', 'y': 'G'},            # RF edge
                7:  {'z': 'Y', 'x': 'O', 'y': 'G'},  # DLF corner
                8:  {'z': 'Y', 'y': 'G'},            # DF edge
                9:  {'z': 'Y', 'X': 'R', 'y': 'G'},  # DRF corner

                # Middle slice (i=1) - IDs 10-18
                10: {'Z': 'W', 'x': 'O'},            # UL edge
                11: {'Z': 'W'},                      # U center
                12: {'Z': 'W', 'X': 'R'},            # UR edge
                13: {'x': 'O'},                      # L center
                14: {},                              # C (cube center)
                15: {'X': 'R'},                      # R center
                16: {'z': 'Y', 'x': 'O'},            # DL edge
                17: {'z': 'Y'},                      # D center
                18: {'z': 'Y', 'X': 'R'},            # DR edge

                # Back face (i=2) - IDs 19-27
                19: {'Z': 'W', 'x': 'O', 'Y': 'Y'},  # ULB corner
                20: {'Z': 'W', 'Y': 'Y'},            # UB edge
                21: {'Z': 'W', 'X': 'R', 'Y': 'Y'},  # URB corner
                22: {'x': 'O', 'Y': 'Y'},            # LB edge
                23: {'Y': 'Y'},                      # B center
                24: {'X': 'R', 'Y': 'Y'},            # RB edge
                25: {'z': 'Y', 'x': 'O', 'Y': 'Y'},  # DLB corner
                26: {'z': 'Y', 'Y': 'Y'},            # DB edge
                27: {'z': 'Y', 'X': 'R', 'Y': 'Y'}   # DRB corner
            }

    @classmethod
    def categorize_ids_over_piece_types(cls):
        """Identifies edge and corner pieces based on orientation markers."""
        edge_ids = []
        corner_ids = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = cls.piece_initial_ids_at_positions[i, j, k]
                    if piece_id in [5, 11, 13, 14, 15, 17, 23]:
                        continue
                    orientation = cls.piece_initial_orientations[i, j, k]
                    if orientation == 'g':
                        edge_ids.append(piece_id)
                    else:
                        corner_ids.append(piece_id)
        return edge_ids, corner_ids
    
    @classmethod
    def categorize_positions_over_piece_types(cls):
        """ Iterate through all positions in the cube and sort their positions into edges and corners """
        edge_positions = []
        corner_positions = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = cls.piece_initial_ids_at_positions[i, j, k]
                    if piece_id in [5, 11, 13, 14, 15, 17, 23]:
                        continue
                    orientation = cls.piece_initial_orientations[i, j, k]
                    if orientation == 'g':
                        edge_positions.append((i, j, k))
                    else:
                        corner_positions.append((i, j, k))
        return edge_positions, corner_positions

    @staticmethod
    def _load_tables_from_json(filenames: list):
        """
        Loads precomputed tables from JSON files and returns them in a dictionary.

        Args:
            filenames: List of JSON filenames containing the precomputed tables

        Returns:
            dict: A dictionary containing loaded tables, with keys: "edge_distances", "corner_distances", "movements".
            Values are the loaded tables, or None if loading failed for a table type.
        """
        tables = {
            "edge_distances": None,
            "corner_distances": None,
            "movements": None
        }

        for filename in filenames:
            try:
                with open(filename, 'r') as f:
                    serializable_table = json.load(f)
                    
                    # Determine which table type this file contains
                    if 'edge' in filename.lower() and 'distance' in filename.lower():
                        tables["edge_distances"] = {}
                        for pair_str, distance in serializable_table.items():
                            pos_tuple = tuple(eval(pair_str))  # Consistent parsing using eval
                            tables["edge_distances"][pos_tuple] = distance
                            
                    elif 'corner' in filename.lower() and 'distance' in filename.lower():
                        tables["corner_distances"] = {}
                        for pair_str, distance in serializable_table.items():
                            pos_tuple = tuple(eval(pair_str))  # Consistent parsing
                            tables["corner_distances"][pos_tuple] = distance
                            
                    elif 'position' in filename.lower() and 'movement' in filename.lower():
                        tables["movements"] = {}
                        for move, position_movements in serializable_table.items():
                            movements = {}
                            for from_pos_str, to_pos_str in position_movements.items():
                                from_pos = tuple(eval(from_pos_str))
                                to_pos = tuple(eval(to_pos_str))
                                movements[from_pos] = to_pos
                            tables["movements"][move] = movements
                            
            except FileNotFoundError:
                print(f"FileNotFoundError: '{filename}' not found.")
            except json.JSONDecodeError:
                print(f"JSONDecodeError: Could not decode JSON from '{filename}'. File may be corrupted.")
            except Exception as e:
                print(f"Error loading '{filename}': {e}")
        
        # Log which tables were successfully loaded
        loaded_tables = [key for key, value in tables.items() if value is not None]
        if loaded_tables:
            print(f"Successfully loaded tables: {', '.join(loaded_tables)}")
        else:
            print("Warning: No tables were successfully loaded")
            
        return tables
    
class CubeTracker(CubeBase):
    def __init__(self):
        CubeBase.initialize()
        self.piece_current_ids_at_positions = copy.deepcopy(CubeBase.piece_initial_ids_at_positions)
        self.piece_current_orientations = copy.deepcopy(CubeBase.piece_initial_orientations)
    
        self.move_map = {
                'L': self._L, 'l': self._l, 'R': self._R, 'r': self._r,
                'F': self._F, 'f': self._f, 'B': self._B, 'b': self._b,
                'U': self._U, 'u': self._u, 'D': self._D, 'd': self._d,
                'N': self._N
        }
        # The uppercase letters are the clockwise moves, and the lowercase letters are the counter-clockwise moves

        self.corner_move_vs_facelet_swap_map = {
            'L': ((1,2),'x'), 'l': ((1,2),'x'), 'R': ((1,2),'x'), 'r': ((1,2),'x'),
            'F': ((0,2),'y'), 'f': ((0,2),'y'), 'B': ((0,2),'y'), 'b': ((0,2),'y'),
            'U': ((0,1),'z'), 'u': ((0,1),'z'), 'D': ((0,1),'z'), 'd': ((0,1),'z'),
            'N': ((0,0),'x')
        }

    def _rotate_face(self, perspective, face_idx, direction):
        """ Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction """
        def change_perspective(cube, perspective, direction):
            if perspective == 0: return cube
            else: return np.rot90(cube, k=direction, axes=(0, perspective))
        # Convert to the desired perspective, rotate the face, then convert back
        self.piece_current_ids_at_positions = change_perspective(self.piece_current_ids_at_positions, perspective, -1)
        self.piece_current_ids_at_positions[face_idx] = np.rot90(self.piece_current_ids_at_positions[face_idx], k=direction, axes=(0, 1))
        self.piece_current_ids_at_positions = change_perspective(self.piece_current_ids_at_positions, perspective, 1)

        self.piece_current_orientations = change_perspective(self.piece_current_orientations, perspective, -1)
        self.piece_current_orientations[face_idx] = np.rot90(self.piece_current_orientations[face_idx], k=direction, axes=(0, 1))
        self.piece_current_orientations = change_perspective(self.piece_current_orientations, perspective, 1)

    def _L(self): self._rotate_face(2, 0, -1)
    def _l(self): self._rotate_face(2, 0,  1)
    def _R(self): self._rotate_face(2, 2,  1)
    def _r(self): self._rotate_face(2, 2, -1)
    def _F(self): self._rotate_face(0, 0, -1)
    def _f(self): self._rotate_face(0, 0,  1)
    def _B(self): self._rotate_face(0, 2,  1)
    def _b(self): self._rotate_face(0, 2, -1)
    def _U(self): self._rotate_face(1, 0, -1)
    def _u(self): self._rotate_face(1, 0,  1)
    def _D(self): self._rotate_face(1, 2,  1)
    def _d(self): self._rotate_face(1, 2, -1)
    def _N(self): pass

    def map_moves_to_inverse_moves(self):
        all_moves = self.move_map.keys()
        moves = []
        inverse_moves = []
        for move in all_moves:
            if move.isupper():
                moves.append(move)
            else:
                inverse_moves.append(move)
        moves.remove('N')
        moves.sort()
        inverse_moves.sort()
        return list(zip(moves, inverse_moves))
        
    def _get_position_of_piece(self, piece_id):
        """Returns the 3D position vector (tuple) of a piece given the piece_id"""
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_ids_at_positions[i, j, k] == piece_id:
                        return (i, j, k)

    def _get_piece_at_position(self, position):
        """Returns the piece ID at a given position (i, j, k)."""
        i, j, k = position
        return self.piece_current_ids_at_positions[i, j, k]
    
    def _get_orientation_of_piece(self, piece_id):
        """Returns the orientation of a piece given its ID."""
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_ids_at_positions[i, j, k] == piece_id:
                        return self.piece_current_orientations[i, j, k]
    
    def _update_edge_orientations(self, move):
        """Updates the orientations of edges based on the move made """
        for edge in self.edge_positions:
            if move in self.movements.keys():
                if edge in self.movements[move].keys():
                    piece_id = self.piece_current_ids_at_positions[edge]
                    piece_initial_position = tuple(np.argwhere(self.piece_initial_ids_at_positions == piece_id)[0])
                    if self.edge_distances[(piece_initial_position, edge)] == self.edge_distances[(piece_initial_position, self.movements[move][edge])]:
                        current_orientation = self.piece_current_orientations[edge]
                        self.piece_current_orientations[edge] = 'g' if current_orientation=='b' else 'b'
    
    def _update_corner_orientations(self, move):
        """Updates the orientations of corners based on the move made """

        def remove(lst, item):
            return [x for x in lst if x != item]
        
        for corner in self.corner_positions:
            if move in self.movements.keys():
                if corner in self.movements[move].keys():
                    current_orientation = list(self.piece_current_orientations[corner])
                    final_corner_orientation = list(self.piece_initial_orientations[CubeBase.movements[move][corner]])
                    reference_orientation = list(self.piece_initial_orientations[corner].lower())
                    reference_constant_facelet_id = self.corner_move_vs_facelet_swap_map[move][1]
                    reference_constant_facelet = reference_orientation.index(reference_constant_facelet_id)
                    corner_constant_facelet = ''.join(current_orientation).lower().index(reference_constant_facelet_id)
                    final_corner_constant_facelet_id = final_corner_orientation[reference_constant_facelet]
                    corner_facelets_to_swap = remove(list(range(0, len(reference_orientation))), corner_constant_facelet)
                    corner_facelet_ids_to_swap = remove(current_orientation, current_orientation[corner_constant_facelet])
                    
                    zipped = list(zip(corner_facelet_ids_to_swap, corner_facelets_to_swap))
                    new_orientation = list(range(0,3))
                    count = 1
                    for id, facelet in zipped:
                        if count == 1:
                            next = final_corner_orientation.pop(reference_orientation.index(id.lower()))
                            final_corner_orientation.remove(final_corner_constant_facelet_id)
                            new_orientation[facelet] = final_corner_orientation[0]
                        else:
                            new_orientation[facelet] = next
                        if count == 2:
                            break
                        count += 1
    
                    new_orientation[corner_constant_facelet] = final_corner_constant_facelet_id
                    new_orientation = ''.join(new_orientation)
                    self.piece_current_orientations[corner] = new_orientation
                
    def apply_moves(self, move_sequence):
        """Applies the moves to the cube state (piece_current_positions and piece_current_orientations)
        Args:
            move_sequence(list/str): ordered set of moves as a list or a string
        """
        if not isinstance(move_sequence, (list, str)):
            raise ValueError("argument to apply_moves must be a list or a string of valid moves")

        if isinstance(move_sequence, str):
            move_sequence = list(move_sequence) # Convert string to list for consistent iteration

        for index, move in enumerate(move_sequence):
            if move in self.move_map:
                self._update_corner_orientations(move)
                self._update_edge_orientations(move)
                self.move_map[move]()
            else:
                raise ValueError(f"Invalid move: '{move}' at index {index}") # More readable error message

In [4]:
class CubeVisualizer:
    def __init__(self, cube_tracker):
        self.cube_tracker = cube_tracker
        self.output = Output()
        
        # Define color scheme for cube faces
        self.color_map = {
            'W': '#FFFFFF',  # White (top)
            'Y': '#FFFF00',  # Yellow (bottom)
            'O': '#800080',  # Orange (left)
            'R': '#FF0000',  # Red (right)
            'G': '#00FF00',  # Green (front)
            'B': '#0000FF',  # Blue (back)
            'X': '#333333',  # Dark gray (inner faces)
        }

        self.initial_color_vs_direction_map = {
            'x': 'O',
            'X': 'R',
            'y': 'G',
            'Y': 'B',
            'z': 'Y',
            'Z': 'W'
        }

        self.material_index = {
            'X': 0, 
            'x': 1, 
            'Z': 2, 
            'z': 3, 
            'y': 4, 
            'Y': 5
        }
        
        # Set up the 3D scene
        self.scene = Scene(background='#F0F0F0')
        self.camera = PerspectiveCamera(
            position=[6, 6, 6], 
            up=[0, 1, 0],
            aspect=1.0,
            fov=35
        )
        
        # Add lighting
        self.setup_lighting()
        
        # Create a group to hold all cube pieces
        self.cube_group = Group()
        self.scene.add(self.cube_group)
        
        # Create the cube pieces
        self.piece_meshes = {}
        self.create_cube_pieces()
        
        # Renderer with orbit controls
        self.renderer = Renderer(
            camera=self.camera,
            scene=self.scene,
            controls=[OrbitControls(controlling=self.camera)],
            width=450,
            height=450,
            antialias=True
        )
        
        # Animation properties
        self.animation_speed = 0.3  # seconds per move
        self.currently_animating = False
        self.move_history = []
        self.max_history = 30
        
        # Create UI controls
        self.create_ui_controls()
    
    def setup_lighting(self):
        """Set up lighting for better 3D rendering"""
        # Ambient light for overall illumination
        ambient = AmbientLight(color='#ffffff', intensity=0.6)
        self.scene.add(ambient)
        
        # Directional lights from different angles for depth
        light_positions = [
            (5, 10, 5),   # Top-front-right
            (-5, 10, -5), # Top-back-left
            (0, -10, 0),  # Bottom
            (10, 0, 0)    # Right
        ]
        
        for position in light_positions:
            light = DirectionalLight(color='#ffffff', position=position, intensity=0.5)
            self.scene.add(light)
    
    def create_cube_pieces(self):
        """Create 3D meshes for each piece of the cube"""
        # Size of each cubelet (slightly smaller than 1 for visual gap)
        size = 0.95
        gap = (1 - size) / 2
        
        # Clear existing pieces
        for child in list(self.cube_group.children):
            self.cube_group.remove(child)
        
        self.piece_meshes = {}
        
        # Iterate through all positions in the cube
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    piece_id = self.cube_tracker.piece_current_ids_at_positions[i, j, k]
                    if piece_id == 14:  # Skip the center piece (optional)
                        continue
                    
                    # Position in 3D space (centered around origin)
                    # Maps cube position to 3D coordinates with origin at cube center
                    x = k - 1
                    y = 1 - j
                    z = 1 - i
                    
                    # Create piece mesh
                    piece_group = self.create_piece_mesh(piece_id, (i, j, k))
                    piece_group.position = [x, y, z]
                    
                    # Store the mesh for future reference
                    self.piece_meshes[piece_id] = {
                        'mesh': piece_group,
                        'position': (i, j, k)
                    }
                    
                    # Add to the scene
                    self.cube_group.add(piece_group)
    
    def create_piece_mesh(self, piece_id, position):
        """Create a mesh for a single cube piece with correct colors"""
        size = 0.9  # Slightly smaller for visual gap between pieces
        piece_group = Group()
        
        # Get the color information for this piece
        color_info = self.cube_tracker.initial_color_map[piece_id]
        
        # Default material for inner sides (dark gray)
        default_material = MeshLambertMaterial(color=self.color_map['X'])
        
        # Materials for each face of the cube piece (right, left, top, bottom, front, back)
        materials = [default_material] * 6
        
        # Map position to face colors
        i, j, k = position
        if k == 2:  # Right face
            materials[0] = MeshLambertMaterial(color=self.color_map['R'])
        if k == 0:  # Left face
            materials[1] = MeshLambertMaterial(color=self.color_map['O'])
        if j == 0:  # Top face
            materials[2] = MeshLambertMaterial(color=self.color_map['W'])
        if j == 2:  # Bottom face
            materials[3] = MeshLambertMaterial(color=self.color_map['Y'])
        if i == 0:  # Front face
            materials[4] = MeshLambertMaterial(color=self.color_map['G'])
        if i == 2:  # Back face
            materials[5] = MeshLambertMaterial(color=self.color_map['B'])
        
        # Create the box geometry
        geometry = BoxGeometry(size, size, size)
        mesh = Mesh(geometry=geometry, material=materials)
        piece_group.add(mesh)
        
        return piece_group
    
    def update_visualization(self, move:chr, affected_piece_positions:list):
        """Update the visualization based on current cube state"""
        # Update positions of all pieces
        
        for piece_id in [piece_id_1 for piece_id_1 in [self.cube_tracker.piece_current_ids_at_positions[position] for position in affected_piece_positions]]:
            for i in range(3):
                for j in range(3):
                    for k in range(3):
                        if piece_id == 14:  # Skip the center piece
                            continue
                        
                        if piece_id in self.piece_meshes:
                            # Calculate 3D position
                            x = k - 1
                            y = 1 - j
                            z = 1 - i
                            
                            # Update position
                            self.piece_meshes[piece_id]['position'] = (i, j, k)
                            self.piece_meshes[piece_id]['mesh'].position = [x, y, z]
                            self.update_piece_colors(piece_id, (i, j, k), move)
        
    def update_piece_colors(self, piece_id, position, move):
        """Update the colors of a piece based on its orientation and position"""
        corner_orientation_before_move = list(self.cube_tracker.piece_current_orientations[position])
        print("corner_orientation_before_move:", corner_orientation_before_move)
        piece_mesh = self.piece_meshes[piece_id]['mesh']
        print("piece_mesh:", piece_mesh)
        # Get the mesh from the piece group
        if len(piece_mesh.children) > 0:
            cube_mesh = piece_mesh.children[0]
        else:
            return  # No mesh to update
        
        if piece_id in self.cube_tracker.corner_ids:
            cube_faces = CubeBase.cube_faces
            corner_initial_colors = []
            corner_initial_orientation = list(self.cube_tracker.piece_initial_orientations[np.argwhere(self.cube_tracker.piece_initial_ids_at_positions == self.cube_tracker._get_piece_at_position[position])])

            print("piece_id:", piece_id)
            print("piece_position_before_move:", position)

            for facelet_id in corner_initial_orientation:
                corner_initial_colors.append(self.initial_color_vs_direction_map[facelet_id])

            print("corner_initial_colors:",corner_initial_colors)
            print("corner_initial_orientation:",corner_initial_orientation)
            print("corner_orientation_before_move:", corner_orientation_before_move)

            reference_id_to_facelet_map = {
                'x' : 0,
                'y' : 1, 
                'z' : 2,
            }
            facelet_to_reference_id_map = {
                0 : 'x',
                1 : 'y', 
                2 : 'z',
            }
            final_colors = list(range(3))
            for u in range(3):
                    final_colors[reference_id_to_facelet_map[corner_orientation_before_move[u].lower()]] = corner_initial_colors[u]

            print("final_colors:", final_colors)

            default_material = MeshLambertMaterial(color=self.color_map['X'])
            materials = [default_material] * 6

            final_corner_facelet_abs_ids = []
            p,q,r  = self.cube_tracker.movements[move][position]
            print("pqr:",p,q,r)
            final_pos_reference_piece_id = self.cube_tracker.piece_initial_ids_at_positions[p,q,r]
            print("final_pos_reference_piece_id", final_pos_reference_piece_id)
            for normal in ['X','x','Y','y','Z','z']:
                if final_pos_reference_piece_id in cube_faces[normal]:
                    final_corner_facelet_abs_ids.append(normal)
            
            print("final_corner_facelet_abs_ids:", final_corner_facelet_abs_ids)
                    
            for idx in range(3):
                materials[self.material_index[final_corner_facelet_abs_ids[idx]]] = MeshLambertMaterial(color=self.color_map[final_colors[idx]])
            cube_mesh.material = materials
            print(cube_mesh.material)
    
    def get_affected_positions(self, move):
        """Determine which pieces are affected by a given move"""
        # Based on the move, determine which pieces will be rotated
        affected_positions = list(self.cube_tracker.movements[move].keys())
        move_vs_center_map = {
            'F' : (0,1,1),
            'f' : (0,1,1),
            'B' : (2,1,1),
            'b' : (2,1,1),
            'L' : (1,1,0),
            'l' : (1,1,0),
            'R' : (1,1,2),
            'r' : (1,1,2),
            'U' : (1,0,1),
            'u' : (1,0,1),
            'D' : (1,2,1),
            'd' : (1,2,1)
        }
        affected_positions.append(move_vs_center_map[move])
        # Remove duplicates and sort the positions
        affected_positions = list(set(affected_positions))
        affected_positions.sort()
        return affected_positions
    
    def get_move_rotation_params(self, move):
        """Get rotation axis and angle for a move"""
        # Define the rotation parameters for each move
        # (axis, angle in radians)
        rotation_params = {
            'L': ([1, 0, 0], -np.pi/2),
            'l': ([1, 0, 0], np.pi/2),
            'R': ([1, 0, 0], np.pi/2),
            'r': ([1, 0, 0], -np.pi/2),
            'F': ([0, 0, 1], -np.pi/2),
            'f': ([0, 0, 1], np.pi/2),
            'B': ([0, 0, 1], np.pi/2),
            'b': ([0, 0, 1], -np.pi/2),
            'U': ([0, 1, 0], -np.pi/2),
            'u': ([0, 1, 0], np.pi/2),
            'D': ([0, 1, 0], np.pi/2),
            'd': ([0, 1, 0], -np.pi/2)
        }
        
        return rotation_params.get(move, ([0, 0, 0], 0))
    
    def animate_move(self, move):
        """Animate a single move with smooth rotation"""
        if self.currently_animating:
            return
            
        self.currently_animating = True
        
        # Update move history
        self.move_history.append(move)
        if len(self.move_history) > self.max_history:
            self.move_history.pop(0)
        self.update_history_display()
        
        # Helper function to convert axis-angle to quaternion representation
        def axis_angle_to_quaternion(axis, angle):
            """Convert axis-angle rotation to quaternion [x,y,z,w]"""
            # Normalize the axis
            axis_norm = np.sqrt(sum(x*x for x in axis))
            if axis_norm > 0:
                x, y, z = [x/axis_norm for x in axis]
            else:
                return [0, 0, 0, 1]  # Identity quaternion for zero rotation
                
            # Calculate quaternion components
            half_angle = angle / 2
            sin_half = np.sin(half_angle)
            
            return [
                x * sin_half,   # qx
                y * sin_half,   # qy
                z * sin_half,   # qz
                np.cos(half_angle)  # qw
            ]
        
        # Set up animation in a separate thread to keep UI responsive
        def animation_thread():
            try:
                # Get rotation parameters for the move
                axis, angle = self.get_move_rotation_params(move)
                
                # Get affected pieces
                affected_positions  = self.get_affected_positions(move)
                print("affected_positions:", affected_positions)
                pieces_to_rotate = []
                
                # Gather affected piece meshes
                for pos in affected_positions:
                    i, j, k = pos
                    if i < 3 and j < 3 and k < 3:  # Ensure indices are in range
                        piece_id = self.cube_tracker.piece_current_ids_at_positions[i, j, k]
                        if piece_id != 14 and piece_id in self.piece_meshes:
                            pieces_to_rotate.append(self.piece_meshes[piece_id]['mesh'])
                
                # Create a temporary group for rotation
                rotation_group = Group()
                self.scene.add(rotation_group)
                
                # Move pieces to the rotation group
                for piece in pieces_to_rotate:
                    if piece in self.cube_group.children:
                        self.cube_group.remove(piece)
                    rotation_group.add(piece)
                
                # Animate the rotation using quaternions
                steps = 10
                for step in range(steps + 1):
                    rotation_angle = angle * step / steps
                    quaternion = axis_angle_to_quaternion(axis, rotation_angle)
                    rotation_group.quaternion = quaternion
                    time.sleep(self.animation_speed / steps)
                

                self.update_visualization(move, affected_positions)
                # Apply the move to the tracker
                self.cube_tracker.apply_moves(move)
                
                # Move pieces back to the cube group
                for piece in list(rotation_group.children):
                    rotation_group.remove(piece)
                    self.cube_group.add(piece)
                
                # Clean up
                self.scene.remove(rotation_group)
            except Exception as e:
                with self.output:
                    print(f"Animation error: {e}")
            finally:
                self.currently_animating = False
        
        # Start animation thread
        threading.Thread(target=animation_thread).start()
    
    def update_history_display(self):
        """Update the move history display"""
        formatted_history = []
        for move in self.move_history:
            if move.islower():
                formatted_history.append(f"{move.upper()}'")
            else:
                formatted_history.append(move)
        
        history_str = " ".join(formatted_history[-20:])  # Show last 20 moves
        with self.output:
            self.history_display.value = f"<b>Move History:</b> {history_str}"
    
    def create_ui_controls(self):
        """Create UI controls for interacting with the cube"""
        # Move buttons
        self.move_buttons = {}
        button_layout = Layout(width='50px', margin='2px')
        
        for move in ['L', 'R', 'F', 'B', 'U', 'D']:
            # Clockwise button
            cw_button = Button(description=move, layout=button_layout, 
                               button_style='primary')
            cw_button.on_click(lambda b, m=move: self.handle_move(m))
            
            # Counter-clockwise button
            ccw_button = Button(description=f"{move}'", layout=button_layout, 
                               button_style='info')
            ccw_button.on_click(lambda b, m=move.lower(): self.handle_move(m))
            
            self.move_buttons[move] = (cw_button, ccw_button)
        
        # Utility buttons
        self.scramble_button = Button(
            description='Scramble', 
            layout=Layout(width='100px', margin='2px'),
            button_style='warning'
        )
        self.scramble_button.on_click(self.handle_scramble)
        
        self.reset_button = Button(
            description='Reset', 
            layout=Layout(width='100px', margin='2px'),
            button_style='danger'
        )
        self.reset_button.on_click(self.handle_reset)
        
        # Algorithm sequence buttons
        self.sequence_buttons = {}
        seq_layout = Layout(width='100px', margin='2px')
        
        common_sequences = {
            'Sexy Move': 'RUru',
            'T Perm': 'RUrFRRuRUFRF',
            'Y Perm': 'FRUruRFrFUrFRuRF'
        }
        
        for name, sequence in common_sequences.items():
            btn = Button(description=name, layout=seq_layout, button_style='success')
            btn.on_click(lambda b, seq=sequence: self.handle_sequence(seq))
            self.sequence_buttons[name] = btn
        
        # Animation speed control
        self.speed_slider = FloatSlider(
            value=0.3,
            min=0.1,
            max=1.0,
            step=0.1,
            description='Speed:',
            continuous_update=False,
            layout=Layout(width='250px')
        )
        self.speed_slider.observe(self.update_speed, names='value')
        
        # Move history display
        self.history_display = HTML(value="<b>Move History:</b>")
        
        # Arrange UI controls
        basic_moves = [
            HBox([self.move_buttons[m][0], self.move_buttons[m][1]]) 
            for m in ['L', 'R', 'F', 'B', 'U', 'D']
        ]
        
        sequences = HBox([self.sequence_buttons[name] for name in common_sequences])
        utilities = HBox([self.scramble_button, self.reset_button])
        
        # Main control panel
        self.controls = VBox([
            HTML("<h3>Rubik's Cube Simulator</h3>"),
            HTML("<b>Basic Moves:</b>"),
            HBox([VBox(basic_moves[:2]), VBox(basic_moves[2:4]), VBox(basic_moves[4:])]),
            HTML("<b>Common Algorithms:</b>"),
            sequences,
            HTML("<b>Utilities:</b>"),
            utilities,
            self.speed_slider,
            self.history_display,
            self.output
        ])
    
    def update_speed(self, change):
        """Update animation speed from slider"""
        self.animation_speed = change.new
    
    def handle_move(self, move):
        """Handle a move button click"""
        if not self.currently_animating:
            self.animate_move(move)
    
    def handle_sequence(self, sequence):
        """Apply a sequence of moves"""
        if self.currently_animating:
            return
        
        def sequence_thread():
            for move in sequence:
                self.handle_move(move)
                # Wait for animation to complete
                while self.currently_animating:
                    time.sleep(0.05)
                time.sleep(0.1)  # Small pause between moves
        
        threading.Thread(target=sequence_thread).start()
    
    def handle_scramble(self, b):
        """Scramble the cube with random moves"""
        if self.currently_animating:
            return
            
        def scramble_thread():
            try:
                # Generate random sequence of 20 moves
                possible_moves = list('LRFBUDlrfbud')
                moves = [np.random.choice(possible_moves) for _ in range(20)]
                
                with self.output:
                    print(f"Scrambling with sequence: {''.join(moves)}")
                
                for move in moves:
                    # Wait if currently animating
                    while self.currently_animating:
                        time.sleep(0.1)
                    
                    # Apply next move
                    self.animate_move(move)
                    
                    # Wait for animation to complete
                    time.sleep(self.animation_speed + 0.1)
            except Exception as e:
                with self.output:
                    print(f"Scramble error: {e}")
        
        threading.Thread(target=scramble_thread).start()
    
    def handle_reset(self, b):
        """Reset the cube to solved state"""
        if self.currently_animating:
            return
        
        # Create a new cube tracker to reset to initial state
        self.cube_tracker = CubeTracker()
        self.move_history = []
        self.update_history_display()
        
        # Update visualization
        self.create_cube_pieces()
    
    def display(self):
        """Return the widget for display"""
        return VBox([
            self.renderer,
            self.controls
        ])


def create_rubiks_cube_visualization():
    """Create and return the Rubik's cube visualization widget"""
    cube_tracker = CubeTracker()
    visualizer = CubeVisualizer(cube_tracker)
    return visualizer.display()

# Display the interactive Rubik's cube widget
cube_widget = create_rubiks_cube_visualization()
display(cube_widget)

affected_positions: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2)]
corner_orientation_before_move: ['x', 'y', 'Z']
piece_mesh: Group(children=(Mesh(geometry=BoxGeometry(depth=0.9, height=0.9, width=0.9), material=(MeshLambertMaterial(alphaMap=None, aoMap=None, color='#333333', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), MeshLambertMaterial(alphaMap=None, aoMap=None, color='#800080', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), MeshLambertMaterial(alphaMap=None, aoMap=None, color='#FFFFFF', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), MeshLambertMaterial(alphaMap=None, aoMap=None, color='#333333', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), MeshLambertMaterial(alphaMap=None, aoMap=None, color='#00FF00', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), MeshLambertMaterial(alphaMap=None, aoM

TraitError: The 'rotation' trait of a PerspectiveCamera instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.

TraitError: The 'rotation' trait of a PerspectiveCamera instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.

TraitError: The 'rotation' trait of a PerspectiveCamera instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.